# Eigenvalores y eigenvectores 

Otro problema crucial de la álgebra lineal, y, por lo tanto, de la álgebra lineal numérica, es encontrar valores y vectores propios, también llamados eigenvalores y eigenvectores ("eigen" = "propio" en alemán).

Recordemos que $\mathbf{v}$ es un eigenvector de la matriz $\mathsf{M}$,  con eigenvalor $\lambda \in \mathbb{C}$,  si

$$\mathsf{M} \cdot \mathbf{v} = \lambda \, \mathbf{v},$$

es decir que la *dirección* de $\mathbf{v}$ es invariante bajo la transformación $\mathsf{M}$, pero la *magnitud* se puede cambiar (o sea, el vector se puede "estirar").

Recordemos que si $\mathbf{v}$ es un eigenvector de $\mathsf{M}$ con eigenvalor $\lambda$, entonces $\alpha \mathbf{v}$ también lo es, para cualquier escalar $\alpha \neq 0$.

¿Por qué son importantes este tipo de problemas? Surgen en problemas físicos como son el encontrar los modos normales de un conjunto de osciladores acoplados, o las energías posibles de un sistema cuántico. En particular, surgen al llevar a cabo una separación de variables en ecuaciones diferenciales parciales.

Por ejemplo, consideremos el problema de encontrar las energías y estados posibles de una partícula libre en una caja finita de longitud $L$. El problema se reduce a encontrar los valores posibles de $E$ y $\psi$ que satisfacen la ecuación de Helmholtz,

$$\nabla^2 \psi = E \psi$$

con condiciones dadas en la frontera, por ejemplo
$\psi(x=0) = \psi(x=L) = 0$.

## El método de potencias 

No pueden existir métodos directos para calcular eigenvalores para matrices generales, ya que los eigenvalores satisfacen la ecuación polinomial $\mathrm{det}(\mathsf{M} - \lambda \mathsf{I}) = 0$, la cual sabemos que no se puede resolver de manera exacta cuando el grado $>4$.

Por lo tanto, debemos considerar métodos iterativos.

El método iterativo más sencillo para calcular un eigenvalor particular de una matriz es el **método de potencias**. Consiste en calcular potencias de la matriz $\mathsf{M}$, aplicadas a un vector $\mathbf{v}$ inicial arbitrario (pero no-cero).

El algoritmo es como sigue: Se escoge un vector inicial arbitrario y se aplica $\mathsf{M}$ a este vector (es decir, se calcula $\mathsf{M} \cdot \mathbf{v}$). Luego se aplica $\mathsf{M}$ al resultado, y así sucesivamente.


**[1]** (i) Escribe una función que implementa este método, para una matriz `M` dada.

In [33]:
function mult_iter(M, v, n)
    for i in 1:n
        v = M*v
    end
    v
end

mult_iter (generic function with 1 method)

In [34]:
A = [1 2
    3 4]
b = [1, 3]
mult_iter(A,b,2)

2-element Array{Int64,1}:
 37
 81

(ii) Considera una matriz $\mathbf{M}$ de $2 \times 2$, e.g. $\begin{pmatrix} 2 & 1 \\ 1 & 1 \end{pmatrix}$. 
Aplica el método. Imprime los resultados después de cada paso. ¿Cuál problema observas?

In [35]:
M = [2 1
    1 1]
mult_iter(M,b,10)

2-element Array{Int64,1}:
 31241
 19308

Que el vector crece mucho :(.

(iii) ¿Cuál es la solución a este problema? Impleméntalo.

Utilizar un vector unitario para que no crezca.

In [36]:
function normalizar(v)
    v_un = v/norm(v)
end

normalizar (generic function with 1 method)

In [51]:
g,f = normalizar([11 9])
sqrt(g^2+f^2)

1.0

In [58]:
function mult_iter2(M, v, n)
    for i in 1:n
        v = M*v
        v = normalizar(v)
    end
    v
end

mult_iter2 (generic function with 1 method)

In [65]:
mult_iter2(M, b, 100)

2-element Array{Float64,1}:
 0.850651
 0.525731

(iv) Calcula a mano los eigenvalores y eigenvectores de tu matriz. ¿Funciona el método? ¿Cuál eigenvector y eigenvalor te da?

In [66]:
eigvals(M)

2-element Array{Float64,1}:
 0.381966
 2.61803 

O haciéndolo realmente a mano da $$ \lambda_{12}= \frac{3 \pm \sqrt{5}}{2} $$

In [67]:
(3+ sqrt(5) )/2

2.618033988749895

In [68]:
(3- sqrt(5) )/2

0.3819660112501051

In [69]:
eigvecs(M)

2×2 Array{Float64,2}:
  0.525731  -0.850651
 -0.850651  -0.525731

(v) Supón que la matriz $\mathsf{M}$ cuenta con una base completa de eigenvectores $\mathbf{v}_i$. Descompone un vector inicial arbitrario $\mathbf{v}$ como una combinación lineal de los eigenvectores, e investiga el resultado de aplicar $\mathsf{M}^n$ a este vector inicial. Así, explica (con papel y pluma) el comportamiento del método.

$$\vec{v} = \sum_{i=1}^n a_{i} \vec{v_i} $$ 
para una matriz de $n\times n$. Al aplicar $\mathbb{M}^n$ s esto tenemos que
$$ \mathbb{M}^n v = \lambda^n \sum_{i=1}^n a_{i} \vec{v_i} $$
por lo que tendremos que al aplicar el método muchas veces tenderá al eigenvalor con el coeficiente $a_i$ más grande.

**[2]** Aplica el método a la matriz de discretización de la partícula libre en una caja. ¿Cuál eigenvalor está calculando? ¿Qué ocurre?

In [3]:
function schrödinger1d(n)
    Ψ = zeros(n,n)
    k = 1/(n-1)
    Ψ[1,:] = Ψ[:,1] = Ψ[:,end] = Ψ[end,:] = 0
    
    for i in 2:n-1
        for j in 2:n-1
            if i == j
                Ψ[i,j] = 1/(-2k^2)
            end
        end
    end
    
    for j in 2:n-1
        for i in 2:n-1
            #Ψ[i,j] = (Ψ[i+1,j] + Ψ[i-1,j] -2Ψ[i,j]) / k^2
            Ψ[i,j] = (Ψ[i+1,j] + Ψ[i-1,j] -2Ψ[i,j])
        end
    end
    Ψ
end

schrödinger1d (generic function with 1 method)

In [4]:
ss = schrödinger1d(10)

10×10 Array{Float64,2}:
 0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0
 0.0  81.0  -40.5    0.0    0.0    0.0    0.0    0.0    0.0  0.0
 0.0  81.0   40.5  -40.5    0.0    0.0    0.0    0.0    0.0  0.0
 0.0  81.0   40.5   40.5  -40.5    0.0    0.0    0.0    0.0  0.0
 0.0  81.0   40.5   40.5   40.5  -40.5    0.0    0.0    0.0  0.0
 0.0  81.0   40.5   40.5   40.5   40.5  -40.5    0.0    0.0  0.0
 0.0  81.0   40.5   40.5   40.5   40.5   40.5  -40.5    0.0  0.0
 0.0  81.0   40.5   40.5   40.5   40.5   40.5   40.5  -40.5  0.0
 0.0  81.0   40.5   40.5   40.5   40.5   40.5   40.5   40.5  0.0
 0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0

In [91]:
using Plots, Interact
gr()

sys:1: MatplotlibDeprecationWarning: The set_axis_bgcolor function was deprecated in version 2.0. Use set_facecolor instead.


Plots.GRBackend()

**[3]** (Opcional) Considera una matriz real y **simétrica**, $\mathsf{S}$. Extiende el método de potencias para calcular *varios* eigenvalores y eigenvectores. [Pista: ¿cuáles propiedades especiales tienen las matrices simétricas, en términos de eigenvalores y eigenvectores?] 

## Facilidades de Julia para álgebra lineal numérica 

Resulta que Julia cuenta con facilidades muy completas para llevar a cabo tareas de álgebra lineal numérica. 
Provee un interfaz fácil de utilizar a librerías estándares como LAPACK.
Para nuestros fines, dos de los más importantes son los siguientes.

### Sistemas de ecuaciones lineales

Para resolver el sistema de ecuaciones lineales $\mathsf{A} \cdot \mathbf{x} = \mathbf{b}$, utilizamos

    A \ b
    
[La notación rara surge así: `b / A` se puede considerar como $b \, A^{-1}$. De la misma forma, `A \ b` denote $A^{-1} \, b$. Nótese que numéricamente **nunca** hay que calcular explícitamente la inversa de una matriz, sino más bien resolvemos directamente un sistema de ecuaciones].

¿Qué algoritmo utiliza? Resulta que es inteligente: intenta escoger el algoritmo más adecuado para un problema dado.

### Problemas de eigenvalores y eigenvectores

El cálculo de eigenvalores y eigenvectores en general es un área complicada. Julia provee, de nuevo, un interfaz sencillo a algoritmos ya probados. La sintaxis es

    eig(A)
    eigvals(A)
    eigvecs(A)

**[4]** Calcula los eigenvalores y eigenvectores de la matriz de discretización del operador de Schrödinger para una partícula libre. Compáralos con los resultados analíticos. ¿Qué observas?

**[5]** Haz lo mismo para el oscilador armónico cuántico.

## Modos normales

**[6]** Considera un sistema compuesto por cuatro carritos de masa $m$, acoplados con resortes sobre una pista recta. El carrito en cada extremo está acoplado mediante un resorte a una pared. Todos los resortes obedecen la ley de Hooke con la misma constante $k$.


(i) Deduce las ecuaciones de movimiento a partir de la segunda ley de Newton y escríbelas en forma matricial. 

(ii) Para resolver el sistema de ecuaciones, proponemos la solución

$$
\vec{x}(t) = \mathrm{Re}(\vec{z}(t)), \quad \vec{z}(t) = \vec{a}e^{i\omega t},
$$
donde $\vec{a}$ es un vector complejo que tiene la información de la amplitud y la fase de las oscilaciones, que dependen de las condiciones iniciales. 

Las **frecuencias normales** del sistema son las raíces positivas de la ecuación característica de la matriz $\mathbb{K} - \omega^2\mathbb{M}$ y los modos normales son los vectores que satisfacen la ecuación de autovalores $(\mathbb{K} - \omega^2\mathbb{M})\vec{a} = 0$. El movimiento del sistema está dado por la superposición de todos los modos normales.

(iii) Calcula las frecuencias normales y los modos normales del sistema.

(iv) Grafica, para cada modo normal, la posición de los carritos como función del tiempo. Describe cada modo normal en términos del movimiento de cada carrito.

(v) Haz una animación del movimiento.